In [13]:
import pandas as pd
import numpy as np

from utils import (
    execute_query,
    create_table_from_df,
    read_sql_file,
    read_json_file
)

# Leer credenciales

In [ ]:
# credenciales = {
#     'host' : "localhost",
#     'dbname' : "dvdrental",
#     'user' : "postgres",
#     'password' : "",
#     'port' : "5432"
# }

credenciales = read_json_file("credentials.json")

# Queries

In [11]:
query = "SELECT * FROM film;"
df_resultado = execute_query(query, credenciales)

In [12]:
sql_query = read_sql_file("queries/ejemplo1.sql")
df_resultado_2 = execute_query(sql_query, credenciales)

# Crear tablas

## Generar datos de prueba

In [17]:
# Generar 100 filas de datos
np.random.seed(42)  # Fijar semilla para reproducibilidad

num_rows = 100

data = {
    "sale_id": np.arange(1, num_rows + 1),  # IDs de 1 a 100
    "customer_id": np.random.randint(100, 110, size=num_rows),  # 10 clientes (100-109)
    "category": np.random.choice(["Electronics", "Clothes", "Books", "Home", "Beauty", "Food"], size=num_rows),
    "amount": np.random.randint(10, 1000, size=num_rows),  # Montos entre 10 y 1000
    "sale_date": pd.date_range(start="2024-01-01", periods=num_rows, freq="D")  # Fechas consecutivas
}

test_data = pd.DataFrame(data)

# Mostrar primeras filas
test_data.head()

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01
1,2,103,Clothes,575,2024-01-02
2,3,107,Electronics,864,2024-01-03
3,4,104,Food,745,2024-01-04
4,5,106,Beauty,234,2024-01-05


In [18]:
create_table_from_df(test_data,'datos_transaccionales',credenciales)

Tabla 'datos_transaccionales' creada exitosamente en PostgreSQL.


In [19]:
execute_query('SELECT * FROM datos_transaccionales limit 1;', credenciales)

,sale_id,customer_id,category,amount,sale_date
0,1,106,Home,229,2024-01-01 00:00:00


# Ejercicios

1. Numerar las ventas de cada cliente.

📌 Enunciado:
Queremos asignar un número de fila a cada venta de un cliente en orden cronológico. Usa ROW_NUMBER() para numerar cada venta de un cliente, ordenándolas por fecha de venta.

2. Calcular el ranking de ventas por categoría.

📌 Enunciado:
Determina el ranking de ventas dentro de cada categoría, ordenando por monto de venta en orden descendente. Usa RANK() para asignar el puesto de cada venta dentro de su categoría.

3.  Comparar el monto de cada venta con la anterior.

📌 Enunciado:
Para cada venta, queremos ver cuánto vendió un cliente en su venta anterior. Usa LAG() para obtener el monto de la venta anterior de cada cliente.

4. Calcular la suma acumulada de ventas por cliente

📌 Enunciado:
Queremos calcular la suma acumulada del monto de ventas por cliente en orden cronológico. Usa SUM() con OVER() para obtener el total acumulado de cada cliente.

5. Obtener el promedio móvil de ventas por categoría

📌 Enunciado:
Calcula el promedio móvil de las últimas 3 ventas dentro de cada categoría. Usa AVG() con ROWS BETWEEN 2 PRECEDING AND CURRENT ROW para calcularlo.

# Respuestas

1. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sale_date) AS sale_number
FROM sales;

```

2. 

```sql

SELECT 
    category, 
    sale_id, 
    amount, 
    RANK() OVER (PARTITION BY category ORDER BY amount DESC) AS sale_rank
FROM sales;

```

3. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    LAG(amount) OVER (PARTITION BY customer_id ORDER BY sale_date) AS previous_sale
FROM sales;


```

4. 

```sql

SELECT 
    customer_id, 
    sale_id, 
    amount, 
    sale_date,
    SUM(amount) OVER (PARTITION BY customer_id ORDER BY sale_date) AS cumulative_sales
FROM sales;

```

5. 

```sql

SELECT 
    category, 
    sale_id, 
    amount, 
    sale_date,
    AVG(amount) OVER (PARTITION BY category ORDER BY sale_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_avg
FROM sales;


```